## 10.7 Transformer
我们在10.6.2节中比较了卷积神经网络(CNN)、循环神经网络(RNN)和自注意力(self-attention)。值得注意的是，自注意力同时具有并行计算和最短的最大路径长度这两个优势。因此，使用自注意力来设计深度架构是很有吸引力的。对比之前依然以来循环神经网络实现输入表示的自注意力模型，transformer模型完全基于注意力机制，没有任何卷积层或循环神经网络层。尽管transformer最初是应用于在文本数据上的序列到序列学习，但现在已经推广到各种现代的深度学习中，例如语言、视觉、语音和强化学习领域。

### 10.7.1 模型
Transformer作为编码器-解码器架构的一个实例，其整体架构图在图10.7.1中展示。正如所见到的，transformer是由编码器和解码器组成的。与图10.4.1中基于Bahdanau注意力实现的序列到序列的学习相比，transformer的编码器和解码器是基于自注意力的模块叠加而成的，源(输入)序列和目标(输出)序列的嵌入(embedding)表示将加上的位置编码(positional encoding)，再分别输入到编码器和解码器中。

<div align=center>
<img src='../../pics/10_7_1.jpeg' width='50%'>
</div>

图10.7.1 中概述了transformer的架构。从宏观角度来看，transformer的编码器是由多个相同的层叠加而成的，每个层都有两个子层(子层表示为sublayer)。第一个子层是多头注意力(multi-head self-attention)汇聚；第二个子层是基于位置的前馈网络(positionwise feed-forward network)。具体来说，在计算编码器的自注意力时，查询、键和值都来自前一个编码器层的输出。受7.6节中残差网络的启发，每个子层都采用了残差连接(residual connection)。在transformer中，对于序列中的任何位置的任何输入$x \in \mathbb R^d$，都要求满足$sublayer(x) \in \mathbb R^d$，以便残差连接满足$x + sublayer(x) \in \mathbb R^d$。在残差连接的加法计算之后，紧接着应用层规范化(layer normalization)。因此，输入序列对应的每个位置，transformer编码器都将输出一个d维表示向量。

Transformer解码器也是由多个相同的层叠加而成的，并且层中使用了残差连接和层规范化。除了编码器中描述的两个子层之外，解码器还在这两个子层之间插入第三个子层，称为编码器-解码器注意力(encoder-decode attention)层。在编码器-解码器注意力中，查询来自前一个解码器层的输出，而键和值来自整个编码器的输出。在解码器注意力中，查询、键和值都来自上一个解码器的输出。但是，解码器中的每个位置只能考虑该位置之前的所有位置。这种掩蔽(masked)注意力保留了自回归(auto-regressive)属性，确保预测仅依赖于已生成的输出词元。

我们已经描述并实现了基于缩放点积多头注意力10.5节和位置编码10.6.3节。接下来，我们将实现transformer模型的剩余部分。

In [1]:
import math
import pandas as pd
import torch
from torch import nn 
from d2l import torch as d2l

### 10.7.2 基于位置的前馈网络
基于位置的前馈网络对序列中的所有位置的表示进行变换时使用的是同一个多层感知机(MLP)，这就是称前馈网络是基于位置的(positionwise)的原因。在下面的实现中，输入X的形状(批量大小，时间步数或序列长度，隐单元数或特征维度)将被一个两层的感知机转换成形状为(批量大小， 时间步数， ffn_num_outputs)的输出张量

In [2]:
class PositionWiseFFN(nn.Module):
    """基于位置的前馈网络"""
    def __init__(self, ffn_num_input, ffn_num_hiddens, ffn_num_outputs,
                **kwargs):
        super(PositionWiseFFN, self).__init__(**kwargs)
        self.dense1 = nn.Linear(ffn_num_input, ffn_num_hiddens)
        self.relu = nn.ReLU()
        self.dense2 = nn.Linear(ffn_num_hiddens, ffn_num_outputs)

    def forward(self, X):
        return self.dense2(self.relu(self.dense1(X)))

下面的例子显示，改变张量的最里层维度的尺寸，会改变成基于位置的前馈网络的输出尺寸。因为用同一个多层感知机对所有位置上的输入进行变换，所以当所有这些位置的输入相同时，它们的输出也是相同的。

In [4]:
ffn = PositionWiseFFN(4, 4, 8)
ffn.eval()
ffn(torch.ones((2, 3, 4)))[0]

tensor([[-0.2356,  0.0560,  0.3449,  0.0886, -0.0436,  0.5142, -0.1457, -0.4246],
        [-0.2356,  0.0560,  0.3449,  0.0886, -0.0436,  0.5142, -0.1457, -0.4246],
        [-0.2356,  0.0560,  0.3449,  0.0886, -0.0436,  0.5142, -0.1457, -0.4246]],
       grad_fn=<SelectBackward0>)

### 10.7.3 残差连接和层规范化
现在让我们关注图10.7.1中的“加法和规范化(add&norm)”组件。正如本节开头所述，这是由残差连接和紧随其后的层规范化组成的。两者都是构建有效的深度架构的关键。

在7.5节中，我们解释了在一个小批量的样本内基于批量规范化对数据进行重新中心化和重新缩放的调整。层规范化和批量规范化的目标相同，但层规范化是基于特征维度进行规范化。尽管批量规范化在计算机视觉中被广泛应用，但在自然语言处理中批量规范化通常不如层规范化的效果好。

以下代码对比不同的维度的层规范化和批量规范化的效果。

In [6]:
ln = nn.LayerNorm(2)
bn = nn.BatchNorm1d(2)
X = torch.tensor([[1, 2], [2, 3]], dtype=torch.float32)
# 在训练模式下计算X的均值和方差
print('Layer norm: ', ln(X), '\nBatch norm: ', bn(X))

Layer norm:  tensor([[-1.0000,  1.0000],
        [-1.0000,  1.0000]], grad_fn=<NativeLayerNormBackward0>) 
Batch norm:  tensor([[-1.0000, -1.0000],
        [ 1.0000,  1.0000]], grad_fn=<NativeBatchNormBackward0>)


现在我们可以使用残差连接和层规范化来实现`AddNorm`类。暂退法也被称作正则化方法使用

In [7]:
class AddNorm(nn.Module):
    """残差连接后进行层规范化"""
    def __init__(self, normalized_shape, dropout, **kwargs):
        super(AddNorm, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)
        self.ln = nn.LayerNorm(normalized_shape)

    def forward(self, X, Y):
        return self.ln(self.dropout(Y) + X)

残差连接要求两个输入的形状相同，以便加法操作后输出张量的形状相同

In [9]:
add_norm = AddNorm([3, 4], 0.5)
add_norm.eval()
add_norm(torch.ones((2, 3, 4)), torch.ones((2, 3, 4))).shape

torch.Size([2, 3, 4])

### 10.7.4 编码器
有了组成transformer编码器的基础组件，现在可以实现编码器中的一个层。下面的`EncoderBlock`类包含两个子层：多头自注意力和基于位置的前馈网络，这两个子层都使用了残差连接和紧随的层规范化。

In [10]:
class EncoderBlock(nn.Module):
    """transformer编码器"""
    def __init__(self, key_size, query_size, value_size, num_hiddens,
                norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
                dropout, use_bias=False, **kwargs):
        super(EncoderBlock, self).__init__(**kwargs)
        self.attention = d2l.MultiHeadAttention(
            key_size, query_size, value_size, num_hiddens, num_heads, dropout,
            use_bias)
        self.addnorm1 = AddNorm(norm_shape, dropout)
        self.ffn = PositionWiseFFN(
            ffn_num_input, ffn_num_hiddens, num_hiddens)
        self.addnorm2 = AddNorm(norm_shape, dropout)

    def forward(self, X, valid_lens):
        Y = self.addnorm1(X, self.attention(X, X, X, valid_lens))
        return self.addnorm2(Y, self.ffn(Y))

正如我们所看到的，transformer编码器中的任何层都不会改变其输入的形状

In [11]:
X = torch.ones((2, 100, 24))
valid_lens = torch.tensor([3, 2])
encoder_blk = EncoderBlock(24, 24, 24, 24, [100, 24], 24, 48, 8, 0.5)
encoder_blk.eval()
encoder_blk(X, valid_lens).shape

torch.Size([2, 100, 24])

在实现下面的transformer编码器的代码中